In [2]:
import requests
import urllib.request
from bs4 import BeautifulSoup
import pandas as pd

# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize

!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

!conda install -c conda-forge folium=0.5.0 --yes # uncomment this line if you haven't completed the Foursquare API lab
import folium # map rendering library

Solving environment: done

## Package Plan ##

  environment location: /home/jupyterlab/conda

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    openssl-1.0.2p             |       h470a237_1         3.1 MB  conda-forge
    certifi-2018.10.15         |        py36_1000         138 KB  conda-forge
    geopy-1.17.0               |             py_0          49 KB  conda-forge
    ca-certificates-2018.10.15 |       ha4d7672_0         135 KB  conda-forge
    conda-4.5.11               |        py36_1000         651 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         4.1 MB

The following NEW packages will be INSTALLED:

    geographiclib:   1.49-py_0            conda-forge
    geopy:           

1st Dataset: Bubble Tea Shop Dataset from Foursquare 

In [3]:
CLIENT_ID = 'QMW0ZEL5J22NLOR1CJTEPO0AY51SDFMUTWDHWWLJZYOCLSSB' # your Foursquare ID
CLIENT_SECRET = 'B0ZL1YBPH3DML5SAGYKDMEM5ZRBWNARUT2G3ES43M2RXLYOR' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

In [4]:
address = 'Detroit, Michigan'

geolocator = Nominatim()
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print(latitude, longitude)

/home/jupyterlab/conda/lib/python3.6/site-packages/geopy/geocoders/osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


42.3315509 -83.0466403


In [5]:
radius = 50000
search_query = 'Bubble Tea'
url = 'https://api.foursquare.com/v2/venues/search?client_id={}&client_secret={}&ll={},{}&v={}&query={}&radius={}'.format(CLIENT_ID, CLIENT_SECRET, latitude, longitude, VERSION, search_query, radius)
url

'https://api.foursquare.com/v2/venues/search?client_id=QMW0ZEL5J22NLOR1CJTEPO0AY51SDFMUTWDHWWLJZYOCLSSB&client_secret=B0ZL1YBPH3DML5SAGYKDMEM5ZRBWNARUT2G3ES43M2RXLYOR&ll=42.3315509,-83.0466403&v=20180605&query=Bubble Tea&radius=50000'

In [6]:
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5bf4ceb9f594df322746b6be'},
 'response': {'venues': [{'id': '4ded19d97d8ba3b6f34e5db5',
    'name': 'Bubble Tea Express',
    'location': {'address': '359 Ouellette Ave',
     'lat': 42.31657124700113,
     'lng': -83.03863480053576,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.31657124700113,
       'lng': -83.03863480053576}],
     'distance': 1792,
     'cc': 'CA',
     'neighborhood': 'c',
     'city': 'Windsor',
     'state': 'MI',
     'country': 'Canada',
     'formattedAddress': ['359 Ouellette Ave', 'Windsor MI', 'Canada']},
    'categories': [{'id': '4bf58dd8d48988d112941735',
      'name': 'Juice Bar',
      'pluralName': 'Juice Bars',
      'shortName': 'Juice Bar',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_v2/food/juicebar_',
       'suffix': '.png'},
      'primary': True}],
    'venuePage': {'id': '82498816'},
    'referralId': 'v-1542770361',
    'hasPerk': False},
   {'id': '4bf5afe09abec9b64e752

In [9]:
# assign relevant part of JSON to venues
venues = results['response']['venues']

# tranform venues into a dataframe
dataframe = json_normalize(venues)
dataframe.head()

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
0,"[{'id': '4bf58dd8d48988d112941735', 'name': 'J...",NaN,NaN,NaN,NaN,NaN,NaN,False,4ded19d97d8ba3b6f34e5db5,359 Ouellette Ave,...,"[359 Ouellette Ave, Windsor MI, Canada]","[{'label': 'display', 'lat': 42.31657124700113...",42.316571,-83.038635,c,NaN,MI,Bubble Tea Express,v-1542770361,82498816
1,"[{'id': '4bf58dd8d48988d1f9941735', 'name': 'F...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bf5afe09abec9b64e7524e8,NaN,...,"[Windsor ON, Canada]","[{'label': 'display', 'lat': 42.3016562081458,...",42.301656,-83.075263,NaN,NaN,ON,Q-tea Bubble Tea,v-1542770361,NaN
2,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f84d71ce4b0f1893677ecbd,NaN,...,"[4th And Main, Royal Oak, MI 48067, United Sta...","[{'label': 'display', 'lat': 42.46108527084407...",42.461085,-83.135281,NaN,48067,MI,Bubble Tea,v-1542770361,NaN
3,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",312595,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/detroit-bub...,False,557b01f7498efae71149dbd9,22821 Woodward Ave,...,"[22821 Woodward Ave, Ferndale, MI 48220, Unite...","[{'label': 'display', 'lat': 42.46105077580841...",42.461051,-83.135156,NaN,48220,MI,Detroit Bubble Tea,v-1542770361,NaN
4,"[{'id': '4bf58dd8d48988d1c7941735', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,53ab5201498e6ff2262a65e5,NaN,...,"[Downtown, Windsor ON, Canada]","[{'label': 'display', 'lat': 42.270053, 'lng':...",42.270053,-82.983487,NaN,NaN,ON,Bubble Tea,v-1542770361,NaN


In [8]:
list(dataframe)

NameError: name 'dataframe' is not defined

In [10]:

# Removing Canadian Locations
dataframe = dataframe.loc[dataframe['location.country']=="United States"]
dataframe

,categories,delivery.id,delivery.provider.icon.name,delivery.provider.icon.prefix,delivery.provider.icon.sizes,delivery.provider.name,delivery.url,hasPerk,id,location.address,...,location.formattedAddress,location.labeledLatLngs,location.lat,location.lng,location.neighborhood,location.postalCode,location.state,name,referralId,venuePage.id
2,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f84d71ce4b0f1893677ecbd,NaN,...,"[4th And Main, Royal Oak, MI 48067, United Sta...","[{'label': 'display', 'lat': 42.46108527084407...",42.461085,-83.135281,NaN,48067,MI,Bubble Tea,v-1542770361,NaN
3,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",312595,/delivery_provider_grubhub_20180129.png,https://fastly.4sqi.net/img/general/cap/,"[40, 50]",grubhub,https://www.grubhub.com/restaurant/detroit-bub...,False,557b01f7498efae71149dbd9,22821 Woodward Ave,...,"[22821 Woodward Ave, Ferndale, MI 48220, Unite...","[{'label': 'display', 'lat': 42.46105077580841...",42.461051,-83.135156,NaN,48220,MI,Detroit Bubble Tea,v-1542770361,NaN
5,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4bbe3b415ce0d13a21d4df57,27622 Middlebelt Rd,...,"[27622 Middlebelt Rd, Farmington Hills, MI 483...","[{'label': 'display', 'lat': 42.497986, 'lng':...",42.497986,-83.337931,NaN,48334,MI,The Little Bubble Tea House,v-1542770361,NaN
6,"[{'id': '52e81612bcbc57f1066b7a0c', 'name': 'B...",NaN,NaN,NaN,NaN,NaN,NaN,False,5b195cd3851de5002c4ae5e9,3395 Auburn Rd,...,"[3395 Auburn Rd (Squirrel Rd), Auburn Hills, M...","[{'label': 'display', 'lat': 42.633858, 'lng':...",42.633858,-83.220856,NaN,48326,MI,"You, Me And Bubble Tea",v-1542770361,NaN
7,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f326d4f19836c91c7d6c00c,300 Renaissance Ctr,...,"[300 Renaissance Ctr, Detroit, MI 48243, Unite...","[{'label': 'display', 'lat': 42.328823, 'lng':...",42.328823,-83.038981,NaN,48243,MI,Coffee Bean and Tea Leaf,v-1542770361,NaN
8,"[{'id': '52f2ab2ebcbc57f1066b8b1b', 'name': 'S...",NaN,NaN,NaN,NaN,NaN,NaN,False,59b6fbb15f68b932e5333e05,Henry St,...,"[Henry St (at Park Ave), Detroit, MI 48201, Un...","[{'label': 'display', 'lat': 42.34024343251506...",42.340243,-83.054913,NaN,48201,MI,Team Store,v-1542770361,NaN
10,"[{'id': '4bf58dd8d48988d16d941735', 'name': 'C...",NaN,NaN,NaN,NaN,NaN,NaN,False,513a7aa7e4b099febc0f680a,71 Garfield St,...,"[71 Garfield St, Detroit, MI 48201, United Sta...","[{'label': 'display', 'lat': 42.35454000150622...",42.354540,-83.061891,NaN,48201,MI,Socra Tea,v-1542770361,NaN
11,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4e4deafcbd41b76bef955171,231 E Grand Blvd,...,"[231 E Grand Blvd, Detroit, MI 48207, United S...","[{'label': 'display', 'lat': 42.35066, 'lng': ...",42.350660,-83.004201,NaN,48207,MI,Nikki's Ginger Tea,v-1542770361,NaN
12,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,4f3f0cd3e4b02a1895bdda51,71 Garfield St,...,"[71 Garfield St (Woodward Ave), Detroit, MI 48...","[{'label': 'display', 'lat': 42.35446352200176...",42.354464,-83.061279,NaN,48201,MI,Socra Tea,v-1542770361,NaN
14,"[{'id': '4bf58dd8d48988d1dc931735', 'name': 'T...",NaN,NaN,NaN,NaN,NaN,NaN,False,524732d811d2ad8b92e5e91f,8019 Agnes St,...,"[8019 Agnes St (Parker), Detroit, MI 48214, Un...","[{'label': 'display', 'lat': 42.35524309867144...",42.355243,-82.996984,NaN,48214,MI,Tarot & Tea,v-1542770361,NaN


In [11]:
df = dataframe[['id','name', 'location.address','location.formattedAddress','location.lat', 'location.lng']]
df.rename(columns = {'id':'ID','name':'Name','location.address':'Address','location.formattedAddress':'Full Address','location.lat':'Latitude','location.lng':'Longitude'},inplace=True)

df


/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/frame.py:3778: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  # Pop these, since the values are in `kwargs` under different names


,ID,Name,Address,Full Address,Latitude,Longitude
2,4f84d71ce4b0f1893677ecbd,Bubble Tea,NaN,"[4th And Main, Royal Oak, MI 48067, United Sta...",42.461085,-83.135281
3,557b01f7498efae71149dbd9,Detroit Bubble Tea,22821 Woodward Ave,"[22821 Woodward Ave, Ferndale, MI 48220, Unite...",42.461051,-83.135156
5,4bbe3b415ce0d13a21d4df57,The Little Bubble Tea House,27622 Middlebelt Rd,"[27622 Middlebelt Rd, Farmington Hills, MI 483...",42.497986,-83.337931
6,5b195cd3851de5002c4ae5e9,"You, Me And Bubble Tea",3395 Auburn Rd,"[3395 Auburn Rd (Squirrel Rd), Auburn Hills, M...",42.633858,-83.220856
7,4f326d4f19836c91c7d6c00c,Coffee Bean and Tea Leaf,300 Renaissance Ctr,"[300 Renaissance Ctr, Detroit, MI 48243, Unite...",42.328823,-83.038981
8,59b6fbb15f68b932e5333e05,Team Store,Henry St,"[Henry St (at Park Ave), Detroit, MI 48201, Un...",42.340243,-83.054913
10,513a7aa7e4b099febc0f680a,Socra Tea,71 Garfield St,"[71 Garfield St, Detroit, MI 48201, United Sta...",42.354540,-83.061891
11,4e4deafcbd41b76bef955171,Nikki's Ginger Tea,231 E Grand Blvd,"[231 E Grand Blvd, Detroit, MI 48207, United S...",42.350660,-83.004201
12,4f3f0cd3e4b02a1895bdda51,Socra Tea,71 Garfield St,"[71 Garfield St (Woodward Ave), Detroit, MI 48...",42.354464,-83.061279
14,524732d811d2ad8b92e5e91f,Tarot & Tea,8019 Agnes St,"[8019 Agnes St (Parker), Detroit, MI 48214, Un...",42.355243,-82.996984


In [12]:
df['Address']=df['Address'].astype(str)
df = df.loc[df['Address']!="nan"]
final=df.reset_index(drop=True)
final


/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.


,ID,Name,Address,Full Address,Latitude,Longitude
0,557b01f7498efae71149dbd9,Detroit Bubble Tea,22821 Woodward Ave,"[22821 Woodward Ave, Ferndale, MI 48220, Unite...",42.461051,-83.135156
1,4bbe3b415ce0d13a21d4df57,The Little Bubble Tea House,27622 Middlebelt Rd,"[27622 Middlebelt Rd, Farmington Hills, MI 483...",42.497986,-83.337931
2,5b195cd3851de5002c4ae5e9,"You, Me And Bubble Tea",3395 Auburn Rd,"[3395 Auburn Rd (Squirrel Rd), Auburn Hills, M...",42.633858,-83.220856
3,4f326d4f19836c91c7d6c00c,Coffee Bean and Tea Leaf,300 Renaissance Ctr,"[300 Renaissance Ctr, Detroit, MI 48243, Unite...",42.328823,-83.038981
4,59b6fbb15f68b932e5333e05,Team Store,Henry St,"[Henry St (at Park Ave), Detroit, MI 48201, Un...",42.340243,-83.054913
5,513a7aa7e4b099febc0f680a,Socra Tea,71 Garfield St,"[71 Garfield St, Detroit, MI 48201, United Sta...",42.354540,-83.061891
6,4e4deafcbd41b76bef955171,Nikki's Ginger Tea,231 E Grand Blvd,"[231 E Grand Blvd, Detroit, MI 48207, United S...",42.350660,-83.004201
7,4f3f0cd3e4b02a1895bdda51,Socra Tea,71 Garfield St,"[71 Garfield St (Woodward Ave), Detroit, MI 48...",42.354464,-83.061279
8,524732d811d2ad8b92e5e91f,Tarot & Tea,8019 Agnes St,"[8019 Agnes St (Parker), Detroit, MI 48214, Un...",42.355243,-82.996984
9,4ac664b1f964a5201fb420e3,Bubble & Bark,686 Livernois St,"[686 Livernois St, Ferndale, MI 48220, United ...",42.452031,-83.142532


Getting the rating for each venue 

In [13]:
rating_df=final[['ID']]
rating_df['Rating']=''
rating_df

/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


,ID,Rating
0,557b01f7498efae71149dbd9,
1,4bbe3b415ce0d13a21d4df57,
2,5b195cd3851de5002c4ae5e9,
3,4f326d4f19836c91c7d6c00c,
4,59b6fbb15f68b932e5333e05,
5,513a7aa7e4b099febc0f680a,
6,4e4deafcbd41b76bef955171,
7,4f3f0cd3e4b02a1895bdda51,
8,524732d811d2ad8b92e5e91f,
9,4ac664b1f964a5201fb420e3,


Getting 19 Results into a list: Needs to rerun on Tue

In [14]:
all_results=[]
for venue_id in rating_df['ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result= requests.get(url).json()
    all_results.append(result)
all_results   

[{'meta': {'code': 200, 'requestId': '5bf4cefedd57976d47501941'},
  'response': {'venue': {'id': '557b01f7498efae71149dbd9',
    'name': 'Detroit Bubble Tea',
    'contact': {'phone': '2482391131', 'formattedPhone': '(248) 239-1131'},
    'location': {'address': '22821 Woodward Ave',
     'lat': 42.46105077580841,
     'lng': -83.13515617211635,
     'labeledLatLngs': [{'label': 'display',
       'lat': 42.46105077580841,
       'lng': -83.13515617211635}],
     'postalCode': '48220',
     'cc': 'US',
     'city': 'Ferndale',
     'state': 'MI',
     'country': 'United States',
     'formattedAddress': ['22821 Woodward Ave',
      'Ferndale, MI 48220',
      'United States']},
    'canonicalUrl': 'https://foursquare.com/v/detroit-bubble-tea/557b01f7498efae71149dbd9',
    'categories': [{'id': '52e81612bcbc57f1066b7a0c',
      'name': 'Bubble Tea Shop',
      'pluralName': 'Bubble Tea Shops',
      'shortName': 'Bubble Tea',
      'icon': {'prefix': 'https://ss3.4sqi.net/img/categories_

Below is the new final code needs to run on Tuesday    

In [15]:
for venue_id in rating_df['ID']:
    url = 'https://api.foursquare.com/v2/venues/{}?client_id={}&client_secret={}&v={}'.format(venue_id, CLIENT_ID, CLIENT_SECRET, VERSION)
    result = requests.get(url).json()
    if "rating" in result['response']['venue']:
        index=int(rating_df[rating_df['ID']== venue_id].index[0])
        rating=result['response']['venue']['rating']
        rating_df.iloc[index,1]=rating   
rating_df

/home/jupyterlab/conda/lib/python3.6/site-packages/pandas/core/indexing.py:189: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)
/home/jupyterlab/conda/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  import sys


,ID,Rating
0,557b01f7498efae71149dbd9,8
1,4bbe3b415ce0d13a21d4df57,
2,5b195cd3851de5002c4ae5e9,
3,4f326d4f19836c91c7d6c00c,
4,59b6fbb15f68b932e5333e05,
5,513a7aa7e4b099febc0f680a,
6,4e4deafcbd41b76bef955171,
7,4f3f0cd3e4b02a1895bdda51,7.6
8,524732d811d2ad8b92e5e91f,
9,4ac664b1f964a5201fb420e3,7.6


In [17]:
full=final.merge(rating_df,left_on='ID', right_on='ID')
full

,ID,Name,Address,Full Address,Latitude,Longitude,Rating
0,557b01f7498efae71149dbd9,Detroit Bubble Tea,22821 Woodward Ave,"[22821 Woodward Ave, Ferndale, MI 48220, Unite...",42.461051,-83.135156,8
1,4bbe3b415ce0d13a21d4df57,The Little Bubble Tea House,27622 Middlebelt Rd,"[27622 Middlebelt Rd, Farmington Hills, MI 483...",42.497986,-83.337931,
2,5b195cd3851de5002c4ae5e9,"You, Me And Bubble Tea",3395 Auburn Rd,"[3395 Auburn Rd (Squirrel Rd), Auburn Hills, M...",42.633858,-83.220856,
3,4f326d4f19836c91c7d6c00c,Coffee Bean and Tea Leaf,300 Renaissance Ctr,"[300 Renaissance Ctr, Detroit, MI 48243, Unite...",42.328823,-83.038981,
4,59b6fbb15f68b932e5333e05,Team Store,Henry St,"[Henry St (at Park Ave), Detroit, MI 48201, Un...",42.340243,-83.054913,
5,513a7aa7e4b099febc0f680a,Socra Tea,71 Garfield St,"[71 Garfield St, Detroit, MI 48201, United Sta...",42.354540,-83.061891,
6,4e4deafcbd41b76bef955171,Nikki's Ginger Tea,231 E Grand Blvd,"[231 E Grand Blvd, Detroit, MI 48207, United S...",42.350660,-83.004201,
7,4f3f0cd3e4b02a1895bdda51,Socra Tea,71 Garfield St,"[71 Garfield St (Woodward Ave), Detroit, MI 48...",42.354464,-83.061279,7.6
8,524732d811d2ad8b92e5e91f,Tarot & Tea,8019 Agnes St,"[8019 Agnes St (Parker), Detroit, MI 48214, Un...",42.355243,-82.996984,
9,4ac664b1f964a5201fb420e3,Bubble & Bark,686 Livernois St,"[686 Livernois St, Ferndale, MI 48220, United ...",42.452031,-83.142532,7.6


Saving the dataframe to excel

In [28]:
full.to_excel('bubble-tea-excel.xlsx')